In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import requests
import time
import pymongo

In [2]:
from splinter import Browser

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\joeki\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


## NASA Mars News

In [4]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [5]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

# news_title = soup.title.text.find('a', .content_title)
# news_paragraph = soup.body.find('p', .article_teaser_body)
element = soup.select_one('ul.item_list li.slide')

news_title = element.find('div', class_="content_title").get_text()
news_title

# for title in news_title:
#     title

'Another First: Perseverance Captures the Sounds of Driving on Mars'

In [6]:
news_paragraph = element.find('div', class_="article_teaser_body").get_text()
news_paragraph


'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.'

## JPL Mars Space Images - Featured Image

In [7]:
url_image = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(url_image)

In [8]:
image_button = browser.find_by_css("button.btn.btn-outline-light")
image_button.click()

In [9]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [10]:
image_link = soup.select_one("div.fancybox-inner img").get('src')
image_link

'image/featured/mars3.jpg'

In [11]:
featured_image_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{image_link}'

In [12]:
featured_image_url


'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

## Mars Facts

In [13]:
mars_db = pd.read_html("https://space-facts.com/mars/")[0]
mars_db

0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers

In [14]:
mars_db.columns=["Mars Description", "Values"]
mars_db = mars_db.set_index("Mars Description")
mars_db

Values
Mars Description                                   
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.39 × 10^23 kg (0.11 Earths)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.38 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                   -87 to -5 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers

In [15]:
#mars_db.to_html("mars.html") 
mars_variable = mars_db.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Values</th>\n    </tr>\n    <tr>\n      <th>Mars Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    

In [16]:
# mars_db.to_html("index.html")

   ## Mars Hemispheres

In [17]:
mars_classes = [
    "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced", 
    "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced",
    "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced",
    "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"]             
              

In [18]:
hemisphere_image_urls=[]
for classes in mars_classes:
    response = requests.get(classes)
    soup = BeautifulSoup(response.text,'html.parser')
    
    base_url = "https://astrogeology.usgs.gov/"

    image_path = soup.find_all('img', class_='wide-image')[0]['src']
    img_url = base_url + image_path

    title = soup.find_all('h2', class_='title')[0].text.split(' Hemisphere')[0]

    hemisphere_image_urls.append({"title":title,"img_url":img_url})

In [19]:
hemisphere_image_urls

[{'title': 'Cerberus',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]